In [13]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
from glob import glob
from natsort import natsorted
import os
import _3dTo2dSetup as D3


In [14]:
#path to image folder and load 3d model
path = r"3dTo2d\totalUncertancy"
img_paths = natsorted(glob(os.path.join(path, "*.png")))
dim = 735
box_dim = LR.BOX_DIM
flip = True

path_real = r"solid_400001-segmentation-solid_400001-label.nrrd"
real_box = D3.get3dFigure(box_dim, path_real)


In [12]:
#read all images and angles
#format -> x_(+/-)**_(+/-)**_angle.png
prim_rots = []
secd_rots = []
imgs = []

for p in img_paths:
  img = cv2.imread(p, 0)
  f = os.path.basename(p)
  prim_a = int(f[2:5])
  secd_a = int(f[6:9])
  prim_rots.append(prim_a)
  secd_rots.append(secd_a)
  imgs.append(img)
  


In [ ]:
#create back-projection for each image
boxes = []

for i in imgs:
  line_coords = LR.createCoordsForLoadedImageFrom3d(i, flip, dim)
  world_coords = LR.performBresenham3D(
      line_coords, LR.DETECTOR_TO_PATIENT, LR.SRC_TO_PATIENT, box_dim)
  box = LR.savePointsWithinSquare(world_coords, box_dim)
  boxes.append(box)
  print("produced box")


In [ ]:
#add rotation to each box 
for i in range(len(boxes)):
  box_rot = LR.rotatePrimSecond(boxes[i], prim_rots[i], secd_rots[i])
  box_rot = np.where(box_rot > 0.5, 1, 0)
  boxes[i] = box_rot
  print("Rotated box ", i)


In [ ]:
#find uncertancy
uncertainty = []
prev_box_sum = boxes[0]
for b in boxes:
  #add box to previous box
  box_overlap= prev_box_sum + b
  box_overlap = np.where(box_overlap == 2, 1, 0)  # save overlap
  prev_box_sum = box_overlap

  #find overlap with original
  box_sum = box_overlap + real_box
  box_sum = np.where(box_sum == 2, 1, 0)  # save overlap
  uncertainty.append(np.sum(box_sum))
  print("box done")

print(uncertainty/np.sum(real_box))


In [ ]:
#render box(es)
x, y, z = LR.boxToWorldCorrds(real_box, box_dim)


In [ ]:
mlab.points3d(x, y, z,
              mode="cube", color=(1, 0, 0.5), scale_factor=1, opacity=1)
mlab.show()
